<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/ExpandRetreat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import math

def expandOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = 100.0 * (inf + top)/ bottom
  if ret> 100.0:
     ret = 100.0
  if ret<0.0:
     ret = 0.0
  return ret

def dailyIncrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [12]:
def retreatOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = inf - ((100 * ( (100-inf) + top)/ bottom) - (100-inf))
  if ret> 100.0:
     ret = 100.0
  if ret<0.0:
     ret = 0.0
  return ret

def dailyDecrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [14]:
retreatOneDay(9.2,2200)

7.3660866155447415

In [10]:
expandOneDay(65.9,2200)

72.6974398271657

In [22]:
def daysToPendingExpansion(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf < 75.0:
    day += 1
    tinf = expandOneDay(tinf, pop)
  return day

def daysToPendingRetreat(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf > 2.5:
    day += 1
    tinf = retreatOneDay(tinf, pop)
  return day

def daysToExpansion(inf: float, pop: int):
  ret = daysToPendingExpansion(inf,pop)
  ret += 1 # pending day
  ret += 6 # active expansion
  return ret

def daysToRetreat(inf: float, pop: int):
  ret = daysToPendingRetreat(inf,pop)
  ret += 1 # pending day
  ret += 6 # active retreat
  ret += 1
  return ret

In [18]:
daysToPendingExpansion(65.9,2200)

2

In [21]:
daysToPendingRetreat(65.9,2200)

15